#                              We Provide Suitable Locations

## Part 1 - Web Scrapping

In this part, we will collect some important data from internet

**For Link of the page: <a href='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai'>Click here<a/>**

In [1]:
import urllib.request

In [2]:
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai')

**Import the BeautifulSoup library so we can parse HTML and XML documents**

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(page, "lxml")

In [5]:
soup.title.string

'List of neighbourhoods in Mumbai - Wikipedia'

**To see all the tables present in the page**

In [6]:
all_tables = soup.find_all('table')

In [7]:
right_table=soup.find('table', class_='wikitable sortable')
#right_table

In [8]:
A=[]
B=[]
C=[]
D=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==4:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))

In [9]:
import pandas as pd

mumbai_neighborhoods = pd.DataFrame(A,columns=['Area'])
mumbai_neighborhoods['Location'] = B
mumbai_neighborhoods['Latitude'] = C
mumbai_neighborhoods['Longitude'] = D

In [10]:
mumbai_neighborhoods.dtypes

Area         object
Location     object
Latitude     object
Longitude    object
dtype: object

In [11]:
mumbai_neighborhoods['Latitude'] = mumbai_neighborhoods['Latitude'].astype('float')
mumbai_neighborhoods['Longitude'] = mumbai_neighborhoods['Longitude'].astype('float')

In [12]:
mumbai_neighborhoods.head()

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala Andheri,",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270


In [13]:
mumbai_neighborhoods.shape

(93, 4)

**We found out 93 areas of Mumbai**

### Now let's found out mumbai coordinates and plot neighborhoods in its map

In [157]:
import folium

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

from geopy.geocoders import Nominatim

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [15]:
address = 'Mumbai, India'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

18.9387711 72.8353355


In [16]:
map_neigh = folium.Map(location=[latitude, longitude], zoom_start=13)
map_neigh

In [17]:
for lat, lon, neighborhood in zip(mumbai_neighborhoods['Latitude'], mumbai_neighborhoods['Longitude'], mumbai_neighborhoods['Area']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neigh)

    
map_neigh 

## Part 2 - Foursquare

Now let's find out all venues in all mentioned areas above

In [158]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [19]:
CLIENT_ID = 'REPHQH1ZB4HATS1S03UMIVN4QT0PLNM0EB0DMZPKELFWTDXJ' # your Foursquare ID
CLIENT_SECRET = 'TU1OYK5MXOKWNPRLQI0CORK4CY3XORQTAVTORKNZ5PX4M0GO' # your Foursquare Secret
VERSION = '20180604'
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: REPHQH1ZB4HATS1S03UMIVN4QT0PLNM0EB0DMZPKELFWTDXJ
CLIENT_SECRET:TU1OYK5MXOKWNPRLQI0CORK4CY3XORQTAVTORKNZ5PX4M0GO


In [20]:
radius = 2000
LIMIT = 30
venues = []

for lat, lon, neighborhood in zip(mumbai_neighborhoods['Latitude'], mumbai_neighborhoods['Longitude'], mumbai_neighborhoods['Area']):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, VERSION, radius, LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
final_dataset = pd.DataFrame(venues)

In [22]:
final_dataset.columns=['Area', 'Latitude', 'Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
final_dataset.shape

(2514, 7)

In [23]:
final_dataset.head()

,Area,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.1293,72.8434,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
1,Amboli,19.1293,72.8434,Merwans Cake shop,19.119300,72.845418,Bakery
2,Amboli,19.1293,72.8434,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant
3,Amboli,19.1293,72.8434,Hard Rock Cafe Andheri,19.135995,72.835335,American Restaurant
4,Amboli,19.1293,72.8434,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant


In [34]:
final_dataset.dtypes

Area                object
Latitude           float64
Longitude          float64
Venue Name          object
Venue Latitude     float64
Venue Longitude    float64
Venue Category      object
dtype: object

In [59]:
final_dataset.isnull().sum()

Area               0
Latitude           0
Longitude          0
Venue Name         0
Venue Latitude     0
Venue Longitude    0
Venue Category     0
dtype: int64

The dataset don't have any null values

In [30]:
Area_grouped_data = final_dataset.groupby(['Area']).count()

In [91]:
Area_grouped_data.head()

,Area,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category,less_than_10
0,Aarey Milk Colony,17,17,17,17,17,17,False
1,Agripada,30,30,30,30,30,30,False
2,Altamount Road,30,30,30,30,30,30,False
3,Amboli,30,30,30,30,30,30,False
4,Amrut Nagar,30,30,30,30,30,30,False


In [46]:
Area_grouped_data['less_than_10'] = Area_grouped_data['Latitude']<=10

This will tell us which Areas have less than 10 stalls or restaurants

In [ ]:
Area_grouped_data = Area_grouped_data.reset_index()

In [61]:
namesof_less_than_10 = [Area_grouped_data['Area'][i] for i in range(len(Area_grouped_data['less_than_10'])) if Area_grouped_data['less_than_10'][i]==True]

In [62]:
namesof_less_than_10

['Dongri\n',
 'Gorai\n',
 'Mahul\n',
 'Naigaon\n',
 'Nalasopara\n',
 'Nehru Nagar\n',
 'Uttan\n',
 'Vikhroli\n']

**These are some areas which have less than 10 stalls/restaurants**

## Part 3 - Analyzing

In [27]:
print('There are {} uniques categories.'.format(len(final_dataset['Venue Category'].unique())))

There are 169 uniques categories.


In [115]:
# one hot encoding
mumbai_onehot = pd.get_dummies(final_dataset[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Area'] = final_dataset['Area'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

print(mumbai_onehot.shape)
mumbai_onehot.head()

(2514, 170)


,Area,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,...,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Women's Store,Zoo
0,Amboli,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Amboli,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Amboli,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amboli,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [117]:
mumbai_grouped = mumbai_onehot.groupby(["Area"]).mean().reset_index()

print(mumbai_grouped.shape)
mumbai_grouped

(92, 170)


,Area,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,...,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Women's Store,Zoo
0,Aarey Milk Colony,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,Agripada,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,Altamount Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,Amboli,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.000000
4,Amrut Nagar,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.033333,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.000000
5,Asalfa,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.033333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
6,Ballard Estate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
7,Bandstand Promenade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
8,Bangur Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
9,Bhandup,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.071429,0.000000,...,0.000000,0.000000,0.000000,0.0,0.107143,0.035714,0.0,0.0,0.000000,0.000000


In [114]:
mumbai_onehot.shape

(2514, 170)

#### Let's print each neighborhood along with the top 5 most common venues

In [118]:
num_top_venues = 5

for hood in mumbai_grouped['Area']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Area'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aarey Milk Colony
----
                  venue  freq
0     Indian Restaurant  0.12
1            Restaurant  0.12
2  Gym / Fitness Center  0.12
3                   Gym  0.06
4           Event Space  0.06


----Agripada
----
               venue  freq
0  Indian Restaurant  0.17
1              Hotel  0.07
2      Deli / Bodega  0.07
3         Restaurant  0.07
4     Cosmetics Shop  0.03


----Altamount Road
----
                  venue  freq
0           Pizza Place  0.07
1     Indian Restaurant  0.07
2  Fast Food Restaurant  0.07
3                   Bar  0.07
4                Bakery  0.07


----Amboli
----
                venue  freq
0   Indian Restaurant  0.10
1  Chinese Restaurant  0.10
2         Pizza Place  0.07
3           Juice Bar  0.07
4         Coffee Shop  0.07


----Amrut Nagar
----
               venue  freq
0  Indian Restaurant  0.13
1             Lounge  0.10
2        Coffee Shop  0.07
3              Diner  0.07
4  Afghan Restaurant  0.03


----Asalfa
----
               v

4     Sandwich Place  0.07


----Indian Institute of Technology Bombay campus
----
                venue  freq
0   Indian Restaurant  0.13
1  Italian Restaurant  0.10
2               Hotel  0.07
3                Café  0.07
4               Plaza  0.03


----Irla
----
                venue  freq
0      Ice Cream Shop  0.13
1  Seafood Restaurant  0.13
2                Café  0.10
3   Indian Restaurant  0.07
4                 Pub  0.07


----Jogeshwari West
----
                      venue  freq
0         Indian Restaurant  0.30
1            Ice Cream Shop  0.13
2        Chinese Restaurant  0.10
3        Seafood Restaurant  0.07
4  Maharashtrian Restaurant  0.03


----Juhu
----
    venue  freq
0   Hotel  0.10
1     Bar  0.10
2  Lounge  0.07
3    Café  0.07
4     Spa  0.07


----Kala Ghoda
----
                  venue  freq
0                  Café  0.10
1  Fast Food Restaurant  0.07
2    Seafood Restaurant  0.07
3           Men's Store  0.03
4    Chinese Restaurant  0.03


----Kalina
----
  

               venue  freq
0               Café  0.17
1                Pub  0.13
2  Indian Restaurant  0.10
3        Pizza Place  0.07
4              Diner  0.07


----Vidyavihar
----
               venue  freq
0  Indian Restaurant  0.13
1       Dessert Shop  0.07
2     Ice Cream Shop  0.07
3         Restaurant  0.07
4        Coffee Shop  0.07


----Vikhroli
----
                    venue  freq
0                    Café   0.2
1         Coworking Space   0.1
2          Clothing Store   0.1
3  Furniture / Home Store   0.1
4       Indian Restaurant   0.1


----Vile Parle
----
    venue  freq
0   Hotel  0.10
1     Bar  0.10
2  Lounge  0.07
3    Café  0.07
4     Spa  0.07


----Virar
----
                       venue  freq
0                     Bakery  0.12
1       Fast Food Restaurant  0.12
2  Indian Chinese Restaurant  0.06
3              Shopping Mall  0.06
4                Pizza Place  0.06


----Walkeshwar
----
               venue  freq
0  Indian Restaurant  0.24
1       Dessert Shop 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [119]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in range(num_top_venues):
    if ind<=2:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    else:    
        columns.append('{}th Most Common Venue'.format(ind+1))

In [132]:
columns

['Area',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

In [134]:
# create a new dataframe
areas_venues_sorted = pd.DataFrame(columns=columns)
areas_venues_sorted['Area'] = mumbai_grouped['Area']

for ind in range(mumbai_grouped.shape[0]):
    areas_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

areas_venues_sorted.head()

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aarey Milk Colony,Gym / Fitness Center,Indian Restaurant,Restaurant,Monument / Landmark,Resort,Fast Food Restaurant,Gym,Dance Studio,Farm,Café
1,Agripada,Indian Restaurant,Hotel,Deli / Bodega,Restaurant,Café,Lounge,Shopping Mall,Scenic Lookout,Middle Eastern Restaurant,Pub
2,Altamount Road,Fast Food Restaurant,Bakery,Pizza Place,Indian Restaurant,Bar,Restaurant,Men's Store,Brewery,Sandwich Place,Scenic Lookout
3,Amboli,Indian Restaurant,Chinese Restaurant,Juice Bar,Pizza Place,Coffee Shop,Fast Food Restaurant,Lounge,Pub,Mediterranean Restaurant,Spa
4,Amrut Nagar,Indian Restaurant,Lounge,Coffee Shop,Diner,Hotel,Gym,Pub,Plaza,Multiplex,Mediterranean Restaurant


## Part 4 - Clustering

In [138]:
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 3, 0, 1, 1, 0, 3, 3])

In [139]:
# add clustering labels
areas_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = mumbai_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(areas_venues_sorted.set_index('Area'), on='Area')

mumbai_merged.head() # check the last columns!

,Area,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400,3.0,Indian Restaurant,Chinese Restaurant,Juice Bar,Pizza Place,Coffee Shop,Fast Food Restaurant,Lounge,Pub,Mediterranean Restaurant,Spa
1,"Chakala Andheri,",Western Suburbs,19.111388,72.860833,1.0,Hotel,Indian Restaurant,Italian Restaurant,Lounge,Chinese Restaurant,Seafood Restaurant,Snack Place,Juice Bar,Fast Food Restaurant,Falafel Restaurant
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373,3.0,Indian Restaurant,Café,Pizza Place,American Restaurant,Multiplex,Donut Shop,Brewery,Residential Building (Apartment / Condo),Falafel Restaurant,Lounge
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210,3.0,Pub,Café,Pizza Place,Indian Restaurant,American Restaurant,Juice Bar,Comfort Food Restaurant,Mediterranean Restaurant,Residential Building (Apartment / Condo),Lounge
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270,3.0,Pub,Lounge,Juice Bar,Pizza Place,Indian Restaurant,Bar,Comfort Food Restaurant,Movie Theater,Fast Food Restaurant,Residential Building (Apartment / Condo)


In [147]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Area'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
       
map_clusters

## Part 5 - Clusters

#### Cluster 1

In [148]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Bandra,Western Suburbs",Pizza Place,Bakery,Café,Tea Room,Scenic Lookout,Hotel Bar,Restaurant,Gourmet Shop,Chinese Restaurant,Cocktail Bar
13,"Bandra,Western Suburbs",Bakery,Indian Restaurant,Park,Gourmet Shop,Deli / Bodega,Café,Pizza Place,Restaurant,Snack Place,Salad Place
14,"Bandra,Western Suburbs",Cupcake Shop,Indian Restaurant,Bakery,Fast Food Restaurant,Seafood Restaurant,Deli / Bodega,Park,Salon / Barbershop,Farmers Market,Scenic Lookout
21,Western Suburbs,Hotel,Bar,Café,Lounge,Spa,Beach,Seafood Restaurant,Breakfast Spot,Spanish Restaurant,Dessert Shop
23,"Kandivali West,Western Suburbs",Fast Food Restaurant,Bar,Coffee Shop,Pub,Italian Restaurant,Pizza Place,Gaming Cafe,Breakfast Spot,Food,Burger Joint
26,"Khar,Western Suburbs",Café,Seafood Restaurant,Indian Restaurant,Deli / Bodega,Arcade,Dessert Shop,Bakery,Park,Modern European Restaurant,Farmers Market
27,"Khar,Western Suburbs",Indian Restaurant,Seafood Restaurant,Bar,Deli / Bodega,Dessert Shop,Lounge,Gym / Fitness Center,Coffee Shop,Café,Modern European Restaurant
28,"Malad,Western Suburbs",Hotel,Indian Restaurant,Multiplex,Vegetarian / Vegan Restaurant,Donut Shop,Fast Food Restaurant,Shopping Mall,Bookstore,Sandwich Place,Breakfast Spot
30,"Sanctacruz,Western Suburbs",Ice Cream Shop,Indian Restaurant,Coffee Shop,Hotel,Dessert Shop,Sandwich Place,Café,Clothing Store,Convenience Store,Outdoors & Recreation
31,"Vasai,Western Suburbs",Boat or Ferry,Bar,Harbor / Marina,Ice Cream Shop,Train Station,Platform,Cocktail Bar,Food,Fast Food Restaurant,Farmers Market


#### Cluster 2

In [149]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Western Suburbs,Hotel,Indian Restaurant,Italian Restaurant,Lounge,Chinese Restaurant,Seafood Restaurant,Snack Place,Juice Bar,Fast Food Restaurant,Falafel Restaurant
5,"Andheri,Western Suburbs",Indian Restaurant,Hotel,Italian Restaurant,Fast Food Restaurant,Restaurant,BBQ Joint,Seafood Restaurant,Chinese Restaurant,Garden,Pub
6,"Andheri,Western Suburbs",Hotel,Indian Restaurant,Coffee Shop,Lounge,Restaurant,Italian Restaurant,Spa,Asian Restaurant,Airport,Bagel Shop
11,"Mira-Bhayandar,Western Suburbs",Restaurant,Indian Restaurant,Beach,Zoo,Food,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
15,"Borivali (West),Western Suburbs",Ice Cream Shop,Indian Restaurant,Gym,Dessert Shop,Café,Fast Food Restaurant,Sandwich Place,Italian Restaurant,Chinese Restaurant,Restaurant
16,"Borivali (West),Western Suburbs",Indian Restaurant,Resort,Water Park,Scenic Lookout,Beach,Zoo,Donut Shop,Fast Food Restaurant,Farmers Market,Farm
17,Western Suburbs,Indian Restaurant,Dessert Shop,Chinese Restaurant,Café,Ice Cream Shop,Food Truck,Gym,Juice Bar,Outdoors & Recreation,Department Store
18,"Goregaon,Western Suburbs",Gym / Fitness Center,Indian Restaurant,Restaurant,Monument / Landmark,Resort,Fast Food Restaurant,Gym,Dance Studio,Farm,Café
20,Western Suburbs,Indian Restaurant,Ice Cream Shop,Chinese Restaurant,Seafood Restaurant,Hotel,Snack Place,Juice Bar,Maharashtrian Restaurant,Fast Food Restaurant,Falafel Restaurant
32,"Vasai,Western Suburbs",Hotel,Motorcycle Shop,Mountain,Ice Cream Shop,Indian Restaurant,Dhaba,Restaurant,Electronics Store,Fast Food Restaurant,Farmers Market


#### Cluster 3

In [150]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"Kurla,Eastern Suburbs",Lake,Food & Drink Shop,Zoo,Food,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Electronics Store


#### Cluster 4

In [151]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 3, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Andheri,Western Suburbs",Indian Restaurant,Chinese Restaurant,Juice Bar,Pizza Place,Coffee Shop,Fast Food Restaurant,Lounge,Pub,Mediterranean Restaurant,Spa
2,"Andheri,Western Suburbs",Indian Restaurant,Café,Pizza Place,American Restaurant,Multiplex,Donut Shop,Brewery,Residential Building (Apartment / Condo),Falafel Restaurant,Lounge
3,"Andheri,Western Suburbs",Pub,Café,Pizza Place,Indian Restaurant,American Restaurant,Juice Bar,Comfort Food Restaurant,Mediterranean Restaurant,Residential Building (Apartment / Condo),Lounge
4,"Andheri,Western Suburbs",Pub,Lounge,Juice Bar,Pizza Place,Indian Restaurant,Bar,Comfort Food Restaurant,Movie Theater,Fast Food Restaurant,Residential Building (Apartment / Condo)
7,"Andheri,Western Suburbs",Café,Pub,Ice Cream Shop,Pizza Place,Indian Restaurant,Seafood Restaurant,Coffee Shop,Mediterranean Restaurant,Recreation Center,Bistro
8,"Andheri,Western Suburbs",Café,Pub,Indian Restaurant,Diner,Pizza Place,Seafood Restaurant,Donut Shop,Juice Bar,South Indian Restaurant,Cocktail Bar
9,"Mira-Bhayandar,Western Suburbs",Pizza Place,Indian Restaurant,Café,Multiplex,Convenience Store,Department Store,Recreation Center,Chinese Restaurant,Coffee Shop,Gym / Fitness Center
10,"Mira-Bhayandar,Western Suburbs",Fast Food Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Café,Chinese Restaurant,Indian Restaurant,Sculpture Garden,Restaurant,Mexican Restaurant,Sandwich Place
19,"Goregaon,Western Suburbs",Multiplex,Clothing Store,Indian Restaurant,Shopping Mall,Ice Cream Shop,Gym,Donut Shop,Food Court,Bookstore,Mexican Restaurant
22,"Kandivali West,Western Suburbs",Café,Coffee Shop,Restaurant,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Sporting Goods Shop,Pub,Food,Playground


#### Cluster 5

In [152]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 4, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,South Mumbai,Beach,Lake,Whisky Bar,Indian Restaurant,Restaurant,Zoo,Food,Fast Food Restaurant,Farmers Market,Farm
